In [1]:
import cv2
import numpy as np
import tensorflow
import keras
from keras import layers as L
import ssl
import os

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
def build_model(input_shape, num_classes=3):
    """ Inputs """
    inputs = L.Input(input_shape)

    """ Backbone """
    backbone = keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_tensor=inputs,
        alpha=1.0
    )
    backbone.trainable = True
    # backbone.summary()

    """ Detection Head """
    x = backbone.output
    x = L.Conv2D(256, kernel_size=1, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.Activation("relu")(x)
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dropout(0.5)(x)
    bbox = L.Dense(4, activation="sigmoid", name="bbox")(x)
    label = L.Dense(num_classes, activation="softmax", name="label")(x)

    """ Model """
    model = keras.models.Model(inputs=[inputs], outputs=[bbox, label])
    return model

In [4]:
def load_model():
    model_path = "./model.weights.h5"
    
    model = build_model((218, 384, 3))
    model.load_weights(model_path)

    return model

In [8]:
def show_video(model):
    vid = cv2.VideoCapture(0)
    
    while True:
        _, frame = vid.read()
        grey = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        _, binary_result = cv2.threshold(grey, 127, 255, cv2.THRESH_BINARY) 
        cv2.imwrite("thing/img.jpg", cv2.resize(binary_result, (384, 218)))
        img = cv2.imread("thing/img.jpg")
        os.remove("thing/img.jpg")

        x = cv2.resize(img, (384, 218))
        x = (x - 127.5) / 127.5
        x = np.expand_dims(x, axis=0)

        pred_bbox, label = model.predict(x, verbose=0)
        
        label_found = ""
        color = (221, 221, 221)
        if label[0][0] >= label[0][1] and label[0][0] >= label[0][2]:
            label_found = "H"
            color = (79, 42, 244)
        elif label[0][1] >= label[0][0] and label[0][1] >= label[0][2]:
            label_found = "S"
            color = (21, 64, 47)
        elif label[0][2] >= label[0][0] and label[0][2] >= label[0][1]:
            label_found = "U"
            color = (197, 73, 245)

        max_confidence = max(label[0][0], label[0][1], label[0][2])
        if not max_confidence >= .85:
            label_found = "/"
            color = (0, 0, 0)

        #print(pred_bbox, label, label_found, max_confidence)

        frame = cv2.resize(frame, (384, 218))
        binary_result = cv2.resize(binary_result, (384, 218))

        
        font_size = 0.5
        text = f"{label_found} - {max_confidence:.3f}"
        cv2.putText(frame, text, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, color, 2)

        cv2.imshow("Normal", frame)
        cv2.imshow("Letter Mask", binary_result)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    vid.release()
    cv2.destroyAllWindows()

In [6]:
def main():
    model = load_model()
    show_video(model)

In [7]:
main()

/var/folders/4w/7mmrnj315pzgcqvpx7n96sf40000gn/T/ipykernel_70212/2947174705.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = keras.applications.MobileNetV2(


[[0.6258691 0.6223937 0.7144055 0.7972417]] [[0.4113012  0.31427068 0.2744281 ]] / 0.4113012
[[0.51111984 0.48892635 0.5681495  0.60872936]] [[0.26815814 0.60864604 0.12319583]] / 0.60864604
[[0.53230524 0.47397274 0.5847491  0.5970644 ]] [[0.26433438 0.58075947 0.15490615]] / 0.58075947
[[0.43728432 0.49491346 0.49408773 0.6166676 ]] [[0.24083121 0.6476747  0.11149409]] / 0.6476747
[[0.5155582  0.45473784 0.56965876 0.565425  ]] [[0.27922803 0.60157806 0.1191939 ]] / 0.60157806
[[0.47227842 0.46654454 0.5218537  0.57437545]] [[0.26765457 0.5956467  0.13669872]] / 0.5956467
[[0.4844384  0.4625351  0.53190917 0.56999874]] [[0.2957793  0.55871177 0.14550899]] / 0.55871177
[[0.46370238 0.47764507 0.511084   0.5834615 ]] [[0.3089591  0.54801923 0.14302178]] / 0.54801923
[[0.5149831  0.49135926 0.57147837 0.60506725]] [[0.30222657 0.5587959  0.13897744]] / 0.5587959
[[0.45159498 0.4834322  0.5035612  0.6034333 ]] [[0.28777272 0.57011086 0.14211641]] / 0.57011086
[[0.43941972 0.49068308 0.49